# Cleaning and preprocessing data

## Why do we need to prepare the data?

Data is almost always "dirty", meaning we need to make sure it is suitable to machine learning models.

Actually a huge part of the job of a data scientist is cleaning and manipulating the data.

<div>
<img src="files/data_science_cleaning.png" width="100%" align='center' source='virgilus'> </div>

<div>
<img src="files/features_preparation.png" width="100%" align='center' source='https://www.orita.ai/blog/clean-customer-data-improves-data-stack'/> </div>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

cols = ['WallMat', # The material the walls are made of
        'Alley', # Alley: Type of alley access to property. Grvl -> Gravel, Pave -> Paved, NA -> No alley access
        'Pesos', # The price of the house in Pesos
        'GrLivArea', # Above grade (ground) living area square feet
        'BedroomAbvGr', # Bedrooms above grade (does NOT include basement bedrooms)
        'KitchenAbvGr', # Kitchens above grade
        'OverallCond', # Rates the overall condition of the house 10 is "Very Excellent" and 1 is "Very Poor"
        'Street', # Type of road access to property Grvl -> Gravel, Pave -> Paved
        'SalePrice', # Target ($)
       ]

df = pd.read_csv('data/iowa_housing.csv',
                  usecols=cols
                )
df.head(2)

## Exploration (very quick)

In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
df.dtypes

## Cleaning data

### Duplicates

When using some models, duplicates lines can lead to "data leakage". Indeed if a sample is both inside the train set and test set, then our test sample isn't really a test anymore. It's a good practice to remove the duplicates before selecting the features.

In [ ]:
df.duplicated().sum()

In [ ]:
df = df.drop_duplicates().reset_index(drop=True) # Let's create a new index and get rid of the "Id" column

In [ ]:
df.head(2)

## Missing values

- Missing values can be challenging to handle because there is no standardized way to represent them. Although since version 2 of Pandas, significant efforts have been made to standardize the representation of different types of missing values.

- There are many reasons to get missing values (bug, bad measures, random events...) in a dataset.

    - Sometimes we just want to get rid of them.
    - Sometimes they actually provide useful informations about a phenomenon.
    - Sometimes we want to replace them by the mean or the median of the Series. But we shouldn't do it if more than about 30% of our Series consists in missing values. Also, keep in mind it creates some noise in the dataset.

In [ ]:
(df.isna().sum() / df.shape[0]) * 100

### WallMat

In [ ]:
df = df.drop(columns='WallMat') # Let's get rid of this empty column

### Alley

Let's take a look at the documentation :

```
Alley: Type of alley access to property

       Grvl	Gravel
       Pave	Paved
       NA 	No alley access
```

So, an empty value is actually a value.

In [ ]:
df['Alley'] = df['Alley'].fillna('NoAlley')

### Pesos

A very small portion of our dataset don't have "Pesos" values. One way to deal with this could be to replace the values with the mean.

#### Replacing values with a function

In [ ]:
df['Pesos'].mean()

In [ ]:
df['Pesos'].fillna(df['Pesos'].mean()) # not an 'inplace' method, so df has not been modified.

#### Replacing using the "SimpleImputer" function from sklearn

This is an "estimator", and it works pretty much the same as a model.

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')
imputer.fit(df[['Pesos']])

In [ ]:
imputer.statistics_ # median

In [ ]:
imputer.transform(df[['Pesos']]) # Outputs the data transformed, missing values have been replaced by the median.

#### Replacing values with a custom function

Let's have a look at the lines with missing values for the "Pesos" column. It's easy, we only have 10 of them.

In [ ]:
df.loc[df['Pesos'].isna()] # We still have the SalePrice in $

In [ ]:
#df.loc[~df['Pesos'].isna()] # To visualize the lines with a value
#df.loc[df['Pesos'].notna()] # Another method

In [ ]:
pesos_missing_values_index = df.loc[df['Pesos'].isna()].index # Let's save the index

In [ ]:
(df['Pesos'] / df['SalePrice'])#.unique() # Let's check how much is one peso.

In [ ]:
df['Pesos'] = df['Pesos'].fillna(df['SalePrice'] * 20)

In [ ]:
df.loc[pesos_missing_values_index] # Making sure everything is right

## Outliers

Just like we saw previously with the missing values, outliers can also occur in a dataset. Maybe the user entered a wrong number, maybe the sensor had a bug, maybe a value has been wrongly encoded.

Having outliers can affect your model performances. It also affects statistics such as the mean or the standard variation.

In [ ]:
#np.int8(89) + np.int8(93) # Example of bad encoding

### Boxplot

In [ ]:
df[['GrLivArea']].boxplot()

In [ ]:
df[['GrLivArea']].min() # Impossible value!

In [ ]:
df['GrLivArea'].argmin() # Get the index

In [ ]:
df.iloc[10]

In [ ]:
(df['GrLivArea'] < 10).sum() # Check how many rows are concerned

In [ ]:
df.loc[(df['GrLivArea'] < 10)]

In [ ]:
#df = df.drop(index=10) # Alternative method
df = df.loc[(df['GrLivArea'] > 10)]

## Feature Scaling

### Why scaling?

- Features with large magnitudes can incorrectly outweight features of small magnitudes.
- Scaling to smaller magnitudes improves computationnal efficiency.
- Increases interpretability of feature coefficients.

### Scaling continuous values

We're going to transform continuois features into a common, smaller range.
Beware, not every numeric value is a continuous value (departement number, id, classes represented with an int, ordinal values...).

### Standardizing

With ```sklearn.preprocessing.StandardScaler```, we can transform a feature so that is has a mean of 0 and a standard deviation of 1.

<font size="6">
$z = \frac{{(x - \text{mean})}}{{\text{std}}}$
</font>

In most large data sets (assuming a normal distribution of data) :

- 99.7% of values lie between -3 and 3 standard deviations,
- 95% between -2 and 2 standard deviations
- 68% between -1 and 1 standard deviations.

If our distribution is bell-shaped, standard scaling is probably the way to go but :

- It does not ensure an exact common range.
- It's sensitive to outliers.
- It can distort relative distances between feature values.

<div>
<img src="files/standardisation.png" width="55%" align='center'> </div>

### Normalizing with Min / Max scaling

An other way to proceed is normalizing the data with ```sklearn.preprocessing.MinMaxScaler```. All values will then be compressed in a fix range from 0 to 1 (by default).

<font size="6">
$X' = \frac{{(X - X_{min})}}{{X_{max} - X_{min}}}$
</font>

The Min-Max Scaling is efficient regardless of distribution but :

- It doesn't reduce the affect of outliers.
- It doesn't correct the skewness of a distribution.

<div>
<img src="files/standardized_and_normalized_data.png" width="85%" align='center'> </div>

## Scaling

### Standardization

In [ ]:
df['Pesos'].plot(kind='hist', bins=20);

In [ ]:
from sklearn.preprocessing import StandardScaler

standardscaler = StandardScaler().fit(df[['Pesos']])
df['Pesos'] = standardscaler.transform(df[['Pesos']])

In [ ]:
standardscaler.mean_

### Min / Max Scaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler

minmaxscaler = MinMaxScaler().fit(df[['GrLivArea']])
df['GrLivArea'] = minmaxscaler.transform(df[['GrLivArea']])

Most of the time, it is ok to mix different types of scaling.

## Dataset Balancing

Most of the time in a dataset, each class is imbalanced.

- Maybe you have more people healthy rather than sick.
- Maybe you have more men than woman
- Etc...

Our model will perform better on the classes it has encountered the most. We can balance data whether if it's X or y (target).

### Balancing strategies

- Oversampling of minority class
- Alternatively, computation of new instances for the minority class
- Undersampling of majority class

<div>
<img src="files/undersampling_oversampling.png" width="85%" align='center' source='https://www.kaggle.com/code/rafjaa/resampling-strategies-for-imbalanced-datasets#t1'> </div>

Which one is better ?

- **Oversampling** : We're going to duplicates samples. But remember, we should do it only after we've done our train / test split. Otherwise we might have data leakage.

- **Undersampling** : In that case we don't have to generate fake data but we're losing some real data that might have been useful.

## Synthetic Minority Oversampling TEchnique (SMOTE)

> SMOTE is an oversampling algorithm that generates new minority instances from existing minority instances - based on linear combinations of existing points.

So we can add new fake data points that look real.

<div>
<img src="files/SMOTE_1.png" width="85%" align='center' source='@rikunert'> </div>

<div>
<img src="files/SMOTE_2.png" width="85%" align='center' source='@rikunert'> </div>

### Usage warning

This technique must be applied only on the training set. The test set exists only in order to provide a score so it must be real data.

## Encoding

Most of the models don't understand any other value than numeric values. Encoding allows us to transform non-numerical data to an equivalent form.

In [ ]:
df['Alley'].unique()

### Integer Encoding

It is the process of replacing a string (usually a class/category) with a number. For instance :

In [ ]:
fruit_df = pd.DataFrame({'Fruit': ['Apple', 'Banana', 'Apple', 'Grape', 'Banana', 'Mango']})
fruit_df.head(6)

In [ ]:
# Let's create a mapping dictionary to encode the Fruit column

mapping_dictionary = {'Apple' : 0, 'Banana': 1, 'Grape': 3, 'Mango': 4}
fruit_df['Integer_encoding'] = fruit_df['Fruit'].map(mapping_dictionary)
fruit_df.head()

### Ordinal encoding

We can also encode ordinal data.

In [ ]:
fruit_df = pd.DataFrame({'Fruit': ['Apple', 'Banana', 'Apple', 'Grape', 'Banana', 'Mango'],
                         'Taste': ['Good', 'Bad', 'Average', 'Good', 'Good', 'Bad']
                        })
fruit_df.head(6)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
ordinal_encoder.fit(fruit_df[["Taste"]])

fruit_df["Taste_encoded"] = ordinal_encoder.transform(fruit_df[["Taste"]])

fruit_df.head(6)

But this doesn't look right. It's because we didn't specify any order so our function used the alphabetical order. Let's fix that:

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder(categories=[["Bad","Average","Good"]]) # Adding a new parameter
ordinal_encoder.fit(fruit_df[["Taste"]])

fruit_df["Taste_encoded"] = ordinal_encoder.transform(fruit_df[["Taste"]])

fruit_df.head(6)

### One hot-encoding

It will create a binary column for each possible category.

#### With sklearn

In [ ]:
from sklearn.preprocessing import OneHotEncoder

fruit_df = pd.DataFrame({'Fruit': ['Apple', 'Banana', 'Apple', 'Grape', 'Banana', 'Mango']})

ohe = OneHotEncoder(sparse_output=False).fit(fruit_df[['Fruit']]) # sparse_output = False so we don't have to deal with a scipy compressed matrix object

print(f"The categories detected by the OneHotEncoder are {ohe.categories_}")
print(f"The column names for the encoded values are {ohe.get_feature_names_out()} \n")
print(ohe.transform(fruit_df[['Fruit']]))

fruit_df[ohe.get_feature_names_out()] = ohe.transform(fruit_df[['Fruit']])
fruit_df.head(6)

#### With Pandas

In [ ]:
fruit_df = pd.DataFrame({'Fruit': ['Apple', 'Banana', 'Apple', 'Grape', 'Banana', 'Mango']})

pd.get_dummies(fruit_df['Fruit'])

#### Difference between One Hot Encoding and Dummy variables

- One Hot Encoding usually means you're encoding all your categories.
- Dummy variables usually means all your categories minus one.

Indeed some models work better if instead of $k$ categories, you have $k-1$ categories. In that case you can set the parameter ```drop_first``` to ```True``` with Pandas, or the parameter ```drop``` in scikitlearn which has different options.

In [ ]:
fruit_df = pd.DataFrame({'Fruit': ['Apple', 'Banana', 'Apple', 'Grape', 'Banana', 'Mango']})

pd.get_dummies(fruit_df['Fruit'], drop_first=True)

### Encoding 'Alley'

In [ ]:
df['Alley'].unique()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False, drop=['NoAlley']).fit(df[['Alley']])

print(ohe.categories_) # Basically the fit is just performing a .unique()

df[ohe.get_feature_names_out()] = ohe.transform(df[['Alley']])

df.head()

## Discretization

"Discretization" refers to the process of converting or partitioning continuous attributes, features or variables to discretized or nominal attributes/features/variables/intervals.

It could help us if we wanted to predict a class and not a number.

In [ ]:
df['SalePriceClass'] = pd.cut(x=df['SalePrice'],
                              bins=[df['SalePrice'].min() - 1,
                                    df['SalePrice'].mean(),
                                    df['SalePrice'].max() + 1], 
                              labels=['cheap', 'expensive'])

df.head()

## Feature Creation

If we understand the dataset, we can create new features in order to improve the performance of the data.
Such as :

- Creating the body mass index ( $height / weight²$)
- Compute the time between two events.
- Categorize dates as weekday, weekend, holidays etc.


##  Feature Selection

How can we select our features? How do we know which column is useful?

Selecting only useful columns makes the training process faster, it reduces the number of dimensions and thus reduce the complexity of the model.

### The curse of Dimensionality

More data doesn't mean that the model will be able to better generalize. Actually it can lower the model performances.

<div>
<img src="files/curse_of_dimensionality.png" width="45%" align='center' source="https://builtin.com/data-science/curse-dimensionality"><br></div>

As the number of features or dimensions grows, the amount of data we need to generalize accurately grows exponentially. If we start from one feature and add a new one, this will cause an increase in dimension space to $4 * 4 = 16$. And then $4 * 4 * 4 = 64$, and so on. So as the dimensions keep on increasing, dimensions space increases exponentially.

<div>
<img src="files/curse_of_dimensionality_2.png" width="65%" align='center' source="https://builtin.com/data-science/curse-dimensionality"><br></div>

In [ ]:
### Feature correlation

import seaborn as sns

sns.heatmap(df.corr(numeric_only=True),
            vmin=-1,
            vmax=1,
            annot=True);

In [ ]:
coef_df = df.corr(numeric_only=True).stack().reset_index().rename(columns={'level_0': 'feature_1', 'level_1': 'feature_2', 0:'coef'})
coef_df['coef'] = np.abs(coef_df['coef']) 
coef_df.loc[coef_df['feature_1'] != coef_df['feature_2']].sort_values(by='coef', ascending=False).head()

### Dropping unwanted columns

Obviously here we have a huge data leak : the 'Pesos' column is the same than the target feature!

In [ ]:
df = df.drop(columns='Pesos')

## Modeling

Let's try to predict a class rather than a number using logistic regression.


In [ ]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Encoding the target
target_encoder = LabelEncoder().fit(df['SalePriceClass']) 
y = target_encoder.transform(df['SalePriceClass'])

# Defining the features
X = df.drop(columns=['SalePrice', 'SalePriceClass', 'Street', 'Alley'])

# Scaling numerical features
# Notice that we already normalize GrLivArea
minmax_scaler = MinMaxScaler()
X[["BedroomAbvGr","KitchenAbvGr","OverallCond"]] = minmax_scaler.fit_transform(X[["BedroomAbvGr","KitchenAbvGr","OverallCond"]])

# Instantiate model
log_reg = LogisticRegression(max_iter=1000) 

# Scoring on multiple folds aka Cross Validation, so no need to
scores = cross_val_score(log_reg, X, y, cv=10)
scores.mean()

## Data leakage !

It seems like a good score. But actually we made two mistakes of data leakage

1. During the scaling stage (Standard Scaler and Normalization), we applied our transformations onto our entire dataset.

    So we computed the mean and the standard variation for both our train and test sets, and a little bit of information has leaked into our test set (inside the cross validation).

1. During the encoding stage, we used the OneHotEncoder for the column 'Alley' on our entire dataset. So it knows all possible variables.

1. In the last cell we used our MinMaxScaler on our entire columns.

1. We removed the outliers from our entire dataset, but in real life maybe our test set can contain outliers.

## Feature permutation

"Permutation feature importance is a model inspection technique that measures the contribution of each feature to a fitted model’s statistical performance on a given tabular dataset. This technique involves randomly shuffling the values of a single feature and observing the resulting degradation of the model’s score. By breaking the relationship between the feature and the target, we determine how much the model relies on such particular feature." [From the sklearn doc.](https://scikit-learn.org/stable/modules/permutation_importance.html)

As this method needs to train the model a great number of times, it can only work if the model is simple.

In [ ]:
from sklearn.inspection import permutation_importance

# Fit model
log_model = LogisticRegression().fit(X, y) 

# Performs Permutation
permutation_score = permutation_importance(log_model, X, y, n_repeats=10) 

# Unstack results showing the decrease in performance after shuffling features
importance_df = pd.DataFrame(np.vstack((X.columns,
                                        permutation_score.importances_mean)).T) 
importance_df.columns=['feature','score decrease']

# Show the important features
importance_df.sort_values(by="score decrease", ascending = False) 

Then we can try to retrain the model removing the useless columns.

In [ ]:
# Selecting the strongest features
strongest_features = X.drop(columns=['Alley_Pave', 'OverallCond'])

# Re-instantiating a Logistic Regression
log_reg = LogisticRegression()

# Average accuracy of the cross-validated model
np.mean(cross_val_score(log_reg, strongest_features, y, cv=10))